# Census data API

Using Census data from ACS 1-Year Detailed Tables
 https://api.census.gov/data/2016/acs/acs1/variables.html
 https://api.census.gov/data/2016/acs/acs1/examples.html
BY state› congressional district	
    https://api.census.gov/data/2016/acs/acs1?get=B01001_001E,NAME&for=congressional%20district:*&key=YOUR_KEY_GOES_HERE	
    https://api.census.gov/data/2016/acs/acs1?get=B01001_001E,NAME&for=congressional%20district:*&in=state:*&key=YOUR_KEY_GOES_HERE	
    https://api.census.gov/data/2016/acs/acs1?get=B01001_001E,NAME&for=congressional%20district:01&in=state:01&key=YOUR_KEY_GOES_HERE

            

### Variable reference
B01001_001E UNWEIGHTED SAMPLE COUNT OF THE POPULATION 
B01002_001E	Estimate!!Median age!!Total
B01002_002E	Estimate!!Median age!!Male
B01002_003E	Estimate!!Median age!!Female

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
from datetime import datetime
from api_keys import api_key

In [2]:
base_url="https://api.census.gov/data/2016/acs/acs1?get="
variables=['B01001_001E']
district_base="&for=congressional%20district:"
district='*'
#base_state='&in=state:'
#state='06'
key='&key='+api_key
url=base_url+variables[0]+district_base+district+key
#print(url)

In [3]:
def getAPIData(purl):
    response = requests.get(purl)
    res=response.json()
    pdf=pd.DataFrame(res)
    return pdf

In [4]:
# set column values as should be instead of being in the first row, drop first row
cdist_df=getAPIData(url)
cdist_df.columns=['population','state','congressional district']
population_df=cdist_df.drop(cdist_df.index[0])
population_df.head()

,population,state,congressional district
1,704457,01,01
2,681621,01,02
3,709482,01,03
4,683273,01,04
5,712529,01,05


In [12]:
#Median Age by congressional district
variables=['B01002_001E,B01002_002E,B01002_003E']
url=base_url
for v in variables:
    url+=v
url+=district_base+district+key
mage_df=getAPIData(url)
mage_df.columns=['median age total','median age male', 'median age female','state','congressional district']
mage_df=mage_df.drop(mage_df.index[0])
mage_df.head()



,median age total,median age male,median age female,state,congressional district
1,39.6,38.3,41.0,01,01
2,38.2,36.7,40.1,01,02
3,38.3,36.6,39.8,01,03
4,40.7,39.7,42.0,01,04
5,40.1,38.4,41.8,01,05


,median age total,median age male,median age female,state,congressional district
1,39.6,38.3,41.0,01,01
2,38.2,36.7,40.1,01,02
3,38.3,36.6,39.8,01,03
4,40.7,39.7,42.0,01,04
5,40.1,38.4,41.8,01,05


In [14]:
# merge and save as csv
popandmage_df=pd.merge(population_df,mage_df, how='inner', on=['state','congressional district'] )
popandmage_df.head()

,population,state,congressional district,median age total,median age male,median age female
0,704457,01,01,39.6,38.3,41.0
1,681621,01,02,38.2,36.7,40.1
2,709482,01,03,38.3,36.6,39.8
3,683273,01,04,40.7,39.7,42.0
4,712529,01,05,40.1,38.4,41.8


In [15]:
outfile='pop_age_byCDistrict.csv'
popandmage_df.to_csv(outfile)